# Libraries

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from torch.optim import Adam
import torch.nn.functional as F
import numpy as np

# for teacher
import random

from sklearn.model_selection import train_test_split

# Side variables
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_type = np.float32
torch_type = torch.float32
batch_size = 64
window_length = 1024 # Data points will hold 1024 tokens of observations (should be 1/4 average song length in tokens, remember to remove outliers)
window_step_size = 32 # Sliding window will move this much each time (higher numbers means less data, but less overfitting to similar data)


# Load Data / Tokenize

In [2]:
# load
data = []
with open("./out.txt", "r") as file:
    data = file.read().splitlines()

print(data[5:10])

['<simul>', 'rest_1/2', 'noteD3_1/2', '</simul>', '<simul>']


In [3]:
# now tokenize
unique_tokens = set(list(data) + ["PAD"])
print(len(unique_tokens))
# mapping
note_to_token = {note: idx for idx, note in enumerate(unique_tokens)}
token_to_note = {idx: note for note, idx in note_to_token.items()}


4195


# Data Preprocessing

In [4]:
end_token = note_to_token["<end_song>"]
pad_token = note_to_token["PAD"]
print(end_token)
print(pad_token)

1749
711


In [5]:
data_inp = [note_to_token[tok] for tok in data]


# Assume already tokenized (this is for transformer, must be adapted for other models)
class MusicDataset(Dataset):
    def __init__(self, data, window_length, step_size):
        self.temp_data = data
        self.window_length = window_length
        self.step_size = step_size
        self.final_data = self.apply_window()

    def apply_window(self):
        train_examples = []
        song_data = []
        cur_seq = []

        # first split data by songs
        for tok in self.temp_data:
            cur_seq.append(tok)
            if(tok == end_token):
                song_data.append(cur_seq)
                cur_seq = []
        
        # then apply sliding windows
        for song in song_data:
            idx = 0
            # create sliding windows
            for start_idx in range(0, len(song) - self.window_length + 1, self.step_size):
                end_idx = start_idx + self.window_length
                idx = end_idx # store most recent end_idx
                train_example = song[start_idx:end_idx] # training of length window_length
                train_examples.append(train_example)
            
            # now add ending and pad (so model can learn how to finish predicting)
            train_example = song[idx:]
            train_example += [0] * (self.window_length - len(train_example))
            train_examples.append(train_example)


        return train_examples

    #def apply_window(self):
    #    # return sliding window data + labels
    #    train_examples = []
    #    # cycle through each window configuration, calculating start index and end index
    #    for start_idx in range(0, len(self.temp_data) - self.window_length + 1, self.step_size):
    #        end_idx = start_idx + self.window_length
    #        train_example = self.temp_data[start_idx:end_idx] # training of length window_length
    #        train_examples.append(train_example)
    #        
    #    return train_examples
    
                

    def __len__(self):
        return len(self.final_data)

    def __getitem__(self, idx):
        window = self.final_data[idx]

        return torch.tensor(window).to(device)
    

dataset = MusicDataset(data_inp, window_length, window_step_size)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

training_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

vocab_len = len(unique_tokens)


# RNN

In [ ]:
# RNN Variables

seq_len = 256
embed_size = 512 # larger embed size may require larger dropout
dropout = 0.2
lr = 1e-4
epochs = 20

In [55]:
class Music_RNN(nn.Module):
    def __init__(self, vocab_len, input_size, hidden_size, dropout):
        super(Music_RNN, self).__init__()
        self.embed = nn.Embedding(vocab_len, input_size)
        self.RNN = nn.RNN(input_size, hidden_size, batch_first=True, dropout=dropout)
        self.ff = nn.Linear(hidden_size, vocab_len) # predicting next input autoregressively

    def forward(self, x):
        # x should be (batch_size, seq_len)
        x = self.embed(x) # (batch_size, seq_len, input_size)
        x, _ = self.RNN(x) # (batch_size, seq_len, hidden_size)
        x_pred = self.ff(x) # batch_size, seq_len, vocab_len)

        return x_pred 

In [56]:
model_rnn = Music_RNN(vocab_len, seq_len, embed_size, dropout).to(device)

/home/dsu/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [57]:
# adapt loss and optimizer as needed
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model_rnn.parameters(), lr=lr)

# train
for epoch in range(epochs):
    # Set to train
    model_rnn.train()
    # keep cumalitive losses
    total_losses = 0.0

    for batch in training_dataloader:
        inp = batch[:, :-1] # grabs all but last
        tgt = batch[:, 1:] # grabs all except first (shifted once)

        optimizer.zero_grad()

        preds = model_rnn.forward(inp) # make sure dimensions line up

        loss = criterion(preds.view(-1, vocab_len), tgt.reshape(-1))
        loss.backward()

        optimizer.step()

        total_losses += loss.item()

    print(f"Epoch: {epoch}, Loss: {total_losses / len(training_dataloader)}")


Epoch: 0, Loss: 4.63392983511655


In [58]:
# Validation

model_rnn.eval()

total_losses = 0.0

# validation
with torch.no_grad():
    for batch in validation_dataloader:
        inp = batch[:, :-1] # grabs all but last
        tgt = batch[:, 1:] # grabs all except first (shifted once)

        preds = model_rnn.forward(inp) # make sure dimensions line up

        loss = criterion(preds.view(-1, vocab_len), tgt.reshape(-1))
        total_losses += loss.item()

    print(f"Loss: {total_losses / len(validation_dataloader)}")

Loss: 4.098278461500656


# LSTM

# Transformer

YAYYYYYYYYY Transformer Time WOOOOOOOOO

In [ ]:
#  Model variables
d_model = 128 # Embed Dim
n_decoder_layers = 4 # lower if not that complex and wanting speedup
n_heads = 4 # Number of Attention Heads
d_ff = 512 # Feed Forward Dimensionality (AIAYN paper reccomends 4 times d_model size)
learning_rate = 1e-5 # Maybe increase?
num_epochs = 1 # Change this later so it doesn't take 10 years to run

In [7]:
# Standard positonal encoding used here, could also try time encoding since notes have different timestamps
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, win_len):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(win_len, d_model) # (win_len, d_model)
        position = torch.arange(0, win_len, dtype=torch_type).unsqueeze(1) # (win_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()*(-torch.log(torch.tensor(10000.0))/d_model)) # (d_model/2)
        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)
        pe = pe.unsqueeze(0) # (1, win_len, d_model)
        self.pe = pe.to(device)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x


# mask
def generate_causal_mask(size):
    # size should be target size
    mask = torch.triu(torch.ones(size, size, device=device), diagonal=1)
    mask = mask.masked_fill(mask == 1, float('-inf'))
    '''
    [0, -inf, -inf, -inf]
    [0,   0,  -inf, -inf]
    [0,   0,    0,  -inf]
    [0,   0,    0,    0 ]
    Yay for triangle masking
    '''
    return mask


In [8]:
# Actual decoder
class MusicTransformer(nn.Module):
    def __init__(self, num_tokens, d_model=d_model, nhead=n_heads, dim_ff=d_ff, win_len=window_length, layers=n_decoder_layers):
        super(MusicTransformer, self).__init__()
        self.d_model = d_model
        self.nheads = nhead
        self.dim_ff = dim_ff
        self.win_len = win_len
        self.layers = layers

        # pre-transformer
        self.tok_embed = nn.Embedding(num_tokens, d_model)
        self.pos_enc = PositionalEncoding(d_model, win_len)
        self.out_proj = nn.Linear(d_model, num_tokens) # final projection for token prediction

        # transformer part
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_ff, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=self.layers)


    def forward(self, x, tgt_mask=None):
        '''
        x is the tokenized notes  # (batch_size, win_len)
        tgt_mask masks the predictions
        '''

        x_seq = self.tok_embed(x) # (batch_size, 1, d_model)
        x_seq = self.pos_enc(x_seq) # (batch_size, 1, d_model)

        decoder_output = self.decoder(x_seq, x_seq, tgt_mask=tgt_mask) # (batch_size, win_len, d_model)
        pred = self.out_proj(decoder_output) # (batch, win_len, num_tokens) 
      
        return pred



In [9]:
my_mask = generate_causal_mask(window_length) # win_len by win_len mask

# Create model
num_tokens = vocab_len
model = MusicTransformer(num_tokens, d_model, n_heads, d_ff, window_length, n_decoder_layers).to(device)

# Use Adam cause he's so cool
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0 # cumalative loss

    # Cycle through each batch
    for batch in training_dataloader:
        inp = batch[:, :-1] # grabs all but last
        tgt = batch[:, 1:] # grabs all except first (shifted once)


        optimizer.zero_grad()
        # Pass through model
        output = model(inp)
        # Determine loss
        loss = criterion(output.view(-1, num_tokens), tgt.reshape(-1))
        # Update weights
        loss.backward()
        optimizer.step()
        # Add to epoch_loss
        epoch_loss += loss.item()

    # Now show average loss for epoch
    avg_epoch_loss = epoch_loss / len(training_dataloader)
    print(f"Epoch: [{epoch+1}/{num_epochs}]   Epoch Average Loss: {avg_epoch_loss}")


Epoch: [1/1]   Epoch Average Loss: 7.350050236671381


In [12]:
model.eval()
epoch_loss = 0 # cumalative loss

# Cycle through each batch
for batch in validation_dataloader:
    inp = batch[:, :-1] # grabs all but last
    tgt = batch[:, 1:] # grabs all except first (shifted once)

    # Pass through model
    output = model(inp)
    # Determine loss
    loss = criterion(output.view(-1, num_tokens), tgt.reshape(-1))
    epoch_loss += loss.item()
    
# Now show average loss for 
avg_epoch_loss = epoch_loss / len(validation_dataloader)
print(f"Validation Loss: {avg_epoch_loss}")

Validation Loss: 6.4589960852334665


In [10]:
# generate music
# for now, generate a fixed length, if we add start and end tokens then we can also do that
def generate_music(model, start_tokens, temperature=1.0, num_to_generate=window_length):
    model.eval()

    predicted_tokens = torch.tensor(start_tokens).clone().detach().unsqueeze(0) # (1, len(start_tokens))
    # iteratively predict then add to start_tokens
    for i in range(num_to_generate):
        with torch.no_grad():
            # grab most recent tokens
            logits = model(predicted_tokens[:, -(window_length):]) # (1, len(predicted_tokens), num_tokens)
            logits = logits[:, -1, :] # grabs last token 
            probs = torch.softmax(logits / temperature, dim=-1) # (1, num_tokens)
            pred_token = torch.multinomial(probs, num_samples=1) # (1, 1)

        predicted_tokens = torch.cat([predicted_tokens, pred_token], dim=1) # (1, len(current sequence length))

    return predicted_tokens.squeeze(0).tolist()




In [11]:
# this just grabs a random batch and performs on it
for batch in validation_dataloader:
    bleh = generate_music(model, batch[0][:4], temperature=1.0)
    notes = [token_to_note[token] for token in bleh]
    print(print(notes))
    break

/tmp/ipykernel_818374/2949307198.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_tokens = torch.tensor(start_tokens).clone().detach().unsqueeze(0) # (1, len(start_tokens))


['noteF5_1/4', 'noteC4_1/2', 'noteC4_1/4', '</simul>', 'noteF2_341/512', 'noteA6_1/3', 'noteE-3_1/10', 'noteF#5_17/256', 'noteF5_7/120', 'noteF6_1', 'noteB1_1/4', 'noteE4_3/4', 'noteB4_2/5', 'noteB-4_2', 'noteC#4_17/480', 'rest_67/240', 'noteC4_51/1024', 'noteF5_2', 'noteF#2_2', 'noteG#3_1/24', 'noteE-6_3', 'noteE-2_1/4', 'noteE5_1/15', 'noteC#7_1/5', 'noteE3_683/1024', 'noteD2_1/2', 'noteF5_1/2', 'noteE-7_3/2', 'noteC3_37/240', 'noteC#1_3/5', 'noteD3_113/1024', 'noteD4_1/20', 'noteB-2_195/1024', 'noteE-6_3/2', 'noteF#3_23/160', 'noteB-3_17/96', 'noteC2_1/4', 'noteE-3_1/15', 'noteG3_2/5', 'noteF1_1/4', 'noteB-6_107/480', 'noteB3_6', 'rest_65/16', 'noteE7_1/3', 'noteC#3_1', 'noteC#4_3/8', 'rest_65/16', 'noteF6_2/5', 'noteE2_85/512', 'noteF3_1/16', 'noteG6_23/160', 'noteE-3_37/240', 'noteE-5_69/1024', 'noteD6_2', 'noteF#2_2/3', 'noteB2_6', 'noteA5_13/96', 'noteG3_7/8', 'noteD3_4', 'noteC5_11/96', 'noteB5_1/16', 'noteG3_1/4', 'noteB3_341/1024', 'noteD1_41/512', 'noteE4_85/512', 'noteG#4_1

In [ ]:
torch.save(model.state_dict(), "music_model.pth")